In [79]:
import minsearch

In [80]:
import json

In [81]:
with open('documents-llm.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [82]:
# (docs_raw)

In [83]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [84]:
documents[0]

{'text': 'Yes, but if you want to receive a certificate, you need to submit your project while we’re still accepting submissions.',
 'section': 'General course-related questions',
 'question': 'I just discovered the course. Can I still join?',
 'course': 'llm-zoomcamp'}

In [85]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [86]:
q = 'The course has already started, can I still enroll?'

In [87]:
index.fit(documents)

In [88]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query=q,
    filter_dict={'course': 'llm-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

# results

In [89]:
import os
from groq import Groq
from dotenv import load_dotenv

In [90]:

load_dotenv()  # This loads from .env file
# api_key = os.getenv("GROQ_API_KEY")
# if not api_key:
#     raise ValueError("GROQ_API_KEY environment variable not set")

# client = Groq(api_key=api_key)


True

In [91]:
client = Groq()

In [92]:
response = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
        {
            "role": "user",
            "content": q
        }
    ],
    
)

In [93]:
response.choices[0].message.content

"It depends on the course and institution. Some courses may allow late enrollment, while others may not. I'd recommend checking with the institution or course provider directly to see if late enrollment is possible. They may have specific deadlines or requirements for late enrollment, so it's best to reach out to them for more information. \n\nYou can try contacting the course administrator, instructor, or admissions office to inquire about the possibility of late enrollment. They can provide you with more details on the enrollment process, any potential penalties or fees, and what you can expect if you're allowed to join the course after it has started."

In [94]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [95]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [96]:
prompt = prompt_template.format(question=q, context=context).strip()

In [97]:
# print(prompt)

In [98]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'llm-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

<!-- # If the CONTEXT doesn't contain the answer, output NONE. -->

In [99]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT: 
    {context}
    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [100]:
def llm(prompt):
    response = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ],
    
)

    return response.choices[0].message.content

In [101]:
def rag(query):
    
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [102]:
# query = "How to configure apache spark?"
query = "When does the course  will start?"
rag(query)

'The course will start in Summer 2025.'

In [103]:
rag("What is saturn cloud?")

'Saturn Cloud is not defined in the provided context, but it is mentioned in several sections, particularly in relation to Open-Source LLMs and as an alternative to other cloud services. It appears to be a cloud platform that can be used for notebook operations and has a cache that can be cleaned out using specific code. However, a clear definition of what Saturn Cloud is, is not provided in the given context.'

In [104]:
documents[0]

{'text': 'Yes, but if you want to receive a certificate, you need to submit your project while we’re still accepting submissions.',
 'section': 'General course-related questions',
 'question': 'I just discovered the course. Can I still join?',
 'course': 'llm-zoomcamp'}

In [105]:
from elasticsearch import Elasticsearch

In [106]:
es_client = Elasticsearch('http://localhost:9200') 

In [107]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

# es_client.indices.create(index=index_name, body=index_settings)

In [108]:
documents[0]

{'text': 'Yes, but if you want to receive a certificate, you need to submit your project while we’re still accepting submissions.',
 'section': 'General course-related questions',
 'question': 'I just discovered the course. Can I still join?',
 'course': 'llm-zoomcamp'}

In [109]:
from tqdm.auto import tqdm

In [110]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/86 [00:00<?, ?it/s]

In [120]:
# query = "When does the course  will start?"
query

'When does the course  will start?'

In [122]:
def elastic_search(query):

    search_query = {
            "size": 5,
            "query": {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query,
                            "fields": ["question^3", "text", "section"],
                            "type": "best_fields"
                        }
                    },
                    "filter": {
                        "term": {
                            "course": "llm-zoomcamp"
                        }
                    }
                }
            }
        }
    
    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append({
            'source': hit['_source'],
            'score': hit['_score']
            })
    
    return result_docs

In [113]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
# elastic_search(query)

In [124]:
elastic_search("token using groq")

[{'source': {'text': 'You can use any LLM platform for your experiments and your project. Also, the homework is designed in such a way that you don’t need to have access to any paid services and can do it locally. However, you would need to adjust the code for that platform. See their documentation pages.',
   'section': 'Module 1: Introduction',
   'question': 'OpenSource: Can I use Groq instead of OpenAI?',
   'course': 'llm-zoomcamp'},
  'score': 12.423037},
 {'source': {'text': "The question asks for the number of tokens in gpt-4o model. tiktoken is a python library that can be used to get the number of tokens. You don't need openai api key to to get the number of tokens. You can use the code provided in the question to get the number of tokens.",
   'section': 'Module 1: Introduction',
   'question': "OpenSource: I am using Groq, and it doesn't provide a tokenizer library based on my research. How can we estimate the number of OpenAI tokens asked in homework question 6?",
   'cour

In [ ]:

rag("How to use tokenizer library?")